# Tutorial 06: Multi-Agent Workflows (Basics)

## 🎯 Learning Objectives
By the end of this notebook, you will:
- Use Agent Framework's **built-in workflow builders** for multi-agent coordination
- Build **sequential workflows** with `SequentialBuilder` (agent chaining)
- Build **concurrent workflows** with `ConcurrentBuilder` (parallel execution)
- Understand when to use each basic workflow pattern
- Learn the foundation for advanced patterns (covered in Tutorial 07)

## 📚 Key Concepts

### Why Multi-Agent Systems?

**Single Agent (Tutorials 1-5):**
- ✅ Simple, one agent handles everything
- ❌ Can become overloaded with too many responsibilities
- ❌ No specialization or parallelization
- Example: General-purpose assistant

**Multi-Agent Workflows:**
- ✅ Multiple specialized agents working together
- ✅ Each agent focuses on their domain expertise
- ✅ Parallel execution for speed
- ✅ Clear separation of concerns
- Example: Flight expert + Hotel expert + Activities expert

### Agent Framework Workflow Patterns

The Agent Framework provides **built-in workflow builders**:

**Basic Patterns (This Tutorial):**

1. **`SequentialBuilder`** - Agents work in sequence (A → B → C)
   - Shared conversation context flows through each agent
   - Each agent builds on previous agent's output
   - Use for: Review/refinement, multi-step tasks

2. **`ConcurrentBuilder`** - Agents work in parallel (A, B, C simultaneously)
   - Fan-out to all agents with same input
   - Fan-in aggregates results from all agents
   - Use for: Independent tasks, gathering multiple perspectives

**Advanced Patterns (Tutorial 07):**

3. **`WorkflowBuilder`** - Custom DAG with executors and edges
   - Full control over workflow graph
   - Custom routing logic and conditions
   - Use for: Complex business logic, conditional flows

4. **`MagenticBuilder`** - AI-powered orchestration
   - LLM creates and manages execution plan
   - Dynamic agent selection and coordination
   - Use for: Complex, unpredictable tasks

### Benefits of Framework Workflows

✅ **Built-in coordination** - No manual orchestration code  
✅ **Automatic aggregation** - Results combined automatically  
✅ **Event streaming** - Track progress in real-time  
✅ **Checkpointing** - Pause/resume long workflows  
✅ **Visualization** - See workflow graphs  
✅ **Production-ready** - Error handling, observability built-in

---

## Step 1: Setup and Imports

In [ ]:
import asyncio
from typing import cast

from agent_framework import (
    # Basic Workflow Builders
    SequentialBuilder,
    ConcurrentBuilder,
    # Workflow Events for tracking progress
    WorkflowOutputEvent,
    AgentRunEvent,
    # Basic types
    ChatMessage,
    Role,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

load_dotenv()
print("✅ Imports successful!")
print("📦 Workflow builders ready: SequentialBuilder, ConcurrentBuilder")
print("💡 For advanced patterns (WorkflowBuilder, MagenticBuilder), see Tutorial 07!")

✅ Imports successful!
📦 Workflow builders ready: SequentialBuilder, ConcurrentBuilder, WorkflowBuilder, MagenticBuilder


## Step 2: Create Specialized Agents

Let's create domain expert agents that we'll coordinate using workflows.

In [ ]:
async def create_travel_agents():
    """
    Create specialized travel planning agents.
    """
    # Note: Using AzureCliCredential without context manager to avoid session closure issues
    # The credential is reused across multiple agent calls
    credential = AzureCliCredential()
    chat_client = AzureAIAgentClient(async_credential=credential)
    
    # Flight Expert
    flight_agent = chat_client.create_agent(
        instructions="""
        You are an expert flight booking specialist. 
        Provide concise, practical flight recommendations considering:
        - Best times to book
        - Airline preferences and quality
        - Connection strategies
        - Price vs convenience tradeoffs
        
        Keep responses brief (2-3 sentences max).
        """,
        name="FlightExpert",
    )
    
    # Hotel Expert
    hotel_agent = chat_client.create_agent(
        instructions="""
        You are an expert hotel booking specialist.
        Provide concise hotel recommendations considering:
        - Best neighborhoods for tourists
        - Value for money
        - Proximity to attractions and transport
        - Hotel quality and amenities
        
        Keep responses brief (2-3 sentences max).
        """,
        name="HotelExpert",
    )
    
    # Activities Expert
    activities_agent = chat_client.create_agent(
        instructions="""
        You are an expert local activities and experiences specialist.
        Provide concise activity recommendations considering:
        - Must-see attractions
        - Local favorites and hidden gems
        - Cultural experiences
        - Food and dining
        
        Keep responses brief (2-3 sentences max).
        """,
        name="ActivitiesExpert",
    )
    
    return flight_agent, hotel_agent, activities_agent

print("✅ Agent factory created")

✅ Agent factory created


## Step 3: Sequential Workflow - SequentialBuilder

The **Sequential Pattern** chains agents together where each agent builds on the previous agent's output.

**Use cases:**
- Writing → Reviewing → Editing
- Research → Analysis → Recommendations
- Draft → Critique → Refinement

The shared conversation flows through each participant sequentially.

In [3]:
async def sequential_workflow_demo():
    """
    Demonstrate SequentialBuilder: agents work in sequence.
    Each agent sees the full conversation history from previous agents.
    """
    print("=== Sequential Workflow Pattern ===")
    print("Agents work in sequence: Flight → Hotel → Activities")
    print("Each agent builds on previous agent's recommendations\n")
    
    flight_agent, hotel_agent, activities_agent = await create_travel_agents()
    
    # Build sequential workflow using SequentialBuilder
    workflow = (
        SequentialBuilder()
        .participants([flight_agent, hotel_agent, activities_agent])
        .build()
    )
    
    print("🚀 Running sequential workflow...\n")
    
    # Run workflow and collect outputs
    outputs: list[list[ChatMessage]] = []
    async for event in workflow.run_stream(
        "Plan a weekend trip to Paris. I want flights, a hotel, and cultural activities."
    ):
        # Track agent progress
        if isinstance(event, AgentRunEvent):
            print(f"   🤖 {event.agent_name} is working...")
        
        # Collect final output
        if isinstance(event, WorkflowOutputEvent):
            outputs.append(cast(list[ChatMessage], event.data))
    
    # Display the final conversation
    if outputs:
        print(f"\n{'='*60}")
        print("📋 FINAL CONVERSATION (Sequential Flow)")
        print(f"{'='*60}\n")
        
        for i, msg in enumerate(outputs[-1], start=1):
            name = msg.author_name or ("User" if msg.role == Role.USER else "Assistant")
            print(f"{'-'*60}")
            print(f"{i}. [{name}]")
            print(f"{'-'*60}")
            print(f"{msg.text}\n")
        
        print(f"{'='*60}")
        print("✅ SEQUENTIAL WORKFLOW COMPLETE!")
        print(f"{'='*60}")
        print("Benefits:")
        print("✅ Each agent built on previous recommendations")
        print("✅ Shared conversation context")
        print("✅ No manual coordination code needed")
        print("✅ Clear sequential flow")

await sequential_workflow_demo()

=== Sequential Workflow Pattern ===
Agents work in sequence: Flight → Hotel → Activities
Each agent builds on previous agent's recommendations

🚀 Running sequential workflow...


📋 FINAL CONVERSATION (Sequential Flow)

------------------------------------------------------------
1. [User]
------------------------------------------------------------
Plan a weekend trip to Paris. I want flights, a hotel, and cultural activities.

------------------------------------------------------------
2. [FlightExpert]
------------------------------------------------------------
Book flights 2-3 months in advance for best weekend rates; prioritize direct flights with Air France for quality and convenience. Choose a centrally located boutique hotel in the Marais or Latin Quarter for easy access to sites. For culture, visit the Louvre, take a Seine river cruise, and explore Montmartre; pre-book tickets to avoid lines.

------------------------------------------------------------
3. [HotelExpert]
-----

## Step 4: Concurrent Workflow - ConcurrentBuilder

The **Concurrent Pattern** runs agents in parallel with automatic fan-out/fan-in.

**Use cases:**
- Gathering multiple perspectives simultaneously
- Independent parallel tasks
- Researcher + Marketer + Legal review (different domains)

All agents receive the same input and work simultaneously.

In [4]:
async def concurrent_workflow_demo():
    """
    Demonstrate ConcurrentBuilder: agents work in parallel.
    All agents receive the same prompt simultaneously.
    Results are aggregated automatically.
    """
    print("=== Concurrent Workflow Pattern ===")
    print("Agents work in parallel: Flight ∥ Hotel ∥ Activities")
    print("All agents receive same input, work simultaneously\n")
    
    flight_agent, hotel_agent, activities_agent = await create_travel_agents()
    
    # Build concurrent workflow using ConcurrentBuilder
    workflow = (
        ConcurrentBuilder()
        .participants([flight_agent, hotel_agent, activities_agent])
        .build()
    )
    
    print("🚀 Running concurrent workflow (all agents in parallel)...\n")
    
    # Run workflow and track events
    agent_tracker = set()
    async for event in workflow.run_stream(
        "I'm planning a 3-day trip to Tokyo. Give me recommendations for flights, hotels, and things to do."
    ):
        # Track which agents are working
        if isinstance(event, AgentRunEvent) and event.agent_name not in agent_tracker:
            agent_tracker.add(event.agent_name)
            print(f"   🤖 {event.agent_name} started (running in parallel)")
    
    # Get final outputs
    events = await workflow.run(
        "I'm planning a 3-day trip to Tokyo. Give me recommendations for flights, hotels, and things to do."
    )
    outputs = events.get_outputs()
    
    # Display aggregated results
    if outputs:
        print(f"\n{'='*60}")
        print("📋 AGGREGATED RESULTS (Concurrent Execution)")
        print(f"{'='*60}\n")
        
        for output in outputs:
            messages: list[ChatMessage] = output
            for i, msg in enumerate(messages, start=1):
                name = msg.author_name or "User"
                print(f"{'-'*60}")
                print(f"{i}. [{name}]")
                print(f"{'-'*60}")
                print(f"{msg.text}\n")
        
        print(f"{'='*60}")
        print("✅ CONCURRENT WORKFLOW COMPLETE!")
        print(f"{'='*60}")
        print("Benefits:")
        print("✅ All agents worked simultaneously (faster!)")
        print("✅ Automatic fan-out to all participants")
        print("✅ Automatic aggregation of results")
        print("✅ No manual parallelization code")

await concurrent_workflow_demo()

=== Concurrent Workflow Pattern ===
Agents work in parallel: Flight ∥ Hotel ∥ Activities
All agents receive same input, work simultaneously

🚀 Running concurrent workflow (all agents in parallel)...


📋 AGGREGATED RESULTS (Concurrent Execution)

------------------------------------------------------------
1. [User]
------------------------------------------------------------
I'm planning a 3-day trip to Tokyo. Give me recommendations for flights, hotels, and things to do.

------------------------------------------------------------
2. [FlightExpert]
------------------------------------------------------------
**Flights:** Book 6–8 weeks in advance for best prices; choose ANA or JAL for quality and direct service if available. Opt for a single connection or nonstop for convenience, balancing layover times with fare savings.

**Hotels:** Stay in Shinjuku, Ginza, or Tokyo Station for easy access—try Park Hotel Tokyo (midrange), Hotel Gracery Shinjuku (modern), or APA Hotels for budget-fr

## 🔍 Workflow Pattern Comparison

### When to Use Each Pattern

| Pattern | Builder | Use Case | Pros | Cons |
|---------|---------|----------|------|------|
| **Sequential** | `SequentialBuilder` | Review/refinement, multi-step tasks | Simple, clear flow | Slower, no parallelism |
| **Concurrent** | `ConcurrentBuilder` | Independent tasks, multiple perspectives | Fast, parallel | No sequential dependencies |

### Real-World Examples

**Sequential:**
- Document: Draft → Review → Edit → Approve
- Research: Gather → Analyze → Summarize → Recommend
- Code: Write → Test → Review → Deploy

**Concurrent:**
- Product Launch: Marketing + Legal + Engineering (parallel reviews)
- Research: Multiple researchers investigate different aspects
- Analysis: Technical + Business + Legal perspectives

### Advanced Patterns (Tutorial 07)

For more complex scenarios, see **Tutorial 07: Advanced Workflows**:
- **WorkflowBuilder** - Custom DAGs, conditional routing, validation gates
- **MagenticBuilder** - AI-powered orchestration, dynamic planning

## 💡 Key Takeaways

### What We Learned

1. **Use Built-in Workflow Builders**
   - Don't write manual coordination code!
   - `SequentialBuilder` for sequential tasks
   - `ConcurrentBuilder` for parallel execution

2. **Sequential Workflows**
   - Agents work in order: A → B → C
   - Each agent sees full conversation history
   - Best for refinement, multi-step processes
   - Simple `.participants([agents]).build()` API

3. **Concurrent Workflows**
   - Agents work in parallel: A ∥ B ∥ C
   - Automatic fan-out and aggregation
   - Best for independent tasks, speed
   - Results combined automatically

4. **Workflows Handle Complexity**
   - Automatic coordination
   - Built-in event streaming
   - Error handling and observability
   - Production-ready patterns

### Production Patterns

```python
# Sequential: Review workflow
workflow = (
    SequentialBuilder()
    .participants([writer, editor, approver])
    .build()
)

# Concurrent: Parallel analysis
workflow = (
    ConcurrentBuilder()
    .participants([technical_analyst, business_analyst, legal_analyst])
    .build()
)

# Run workflow
async for event in workflow.run_stream("your prompt"):
    if isinstance(event, WorkflowOutputEvent):
        print(event.data)
```

## 🎯 Practice Exercises

1. **Content Creation Pipeline**
   - Sequential: Idea Generator → Writer → Editor → SEO Optimizer
   - Each agent refines the content

2. **Product Analysis**
   - Concurrent: Technical Reviewer ∥ Market Analyst ∥ Competitor Researcher
   - Get multiple perspectives simultaneously

3. **Code Review**
   - Sequential: Linter → Security Checker → Performance Analyzer → Approver
   - Each step builds on previous checks

4. **Customer Feedback Analysis**
   - Concurrent: Sentiment Analyst ∥ Feature Extractor ∥ Priority Scorer
   - Analyze different aspects in parallel

In [ ]:
# Exercise: Create a content creation workflow

async def content_creation_exercise():
    """
    Create a sequential workflow for content creation:
    Idea Generator → Writer → Editor → SEO Optimizer
    """
    # Your implementation here!
    # 1. Create 4 specialized agents
    # 2. Build sequential workflow with SequentialBuilder
    # 3. Run with a topic
    # 4. Display the final refined content
    pass

async def product_analysis_exercise():
    """
    Create a concurrent workflow for product analysis:
    Technical ∥ Market ∥ Competitor (parallel)
    """
    # Your implementation here!
    # 1. Create 3 specialized analyst agents
    # 2. Build concurrent workflow with ConcurrentBuilder
    # 3. Run with a product description
    # 4. Display aggregated analysis
    pass

print("💡 Exercises ready - implement sequential and concurrent workflows!")

## 🚀 What's Next?

Congratulations! You've mastered basic multi-agent workflow patterns!

You now know how to:
- ✅ Build sequential agent workflows (chaining)
- ✅ Create concurrent parallel execution
- ✅ Use built-in workflow builders
- ✅ Track workflow progress with events

**In Tutorial 07: Advanced Workflows**, you'll learn:
- Build custom workflow graphs with WorkflowBuilder
- Create custom executors with business logic
- Implement conditional routing and validation gates
- Use AI-powered orchestration with MagenticBuilder
- Handle complex multi-stage workflows

**Then in Tutorial 08: Human-in-the-Loop**, you'll learn:
- Pause workflows for human approval
- Implement approval gates
- Build interactive workflows
- Handle human feedback in agent systems

---

### Quick Reference

**Sequential Workflow:**
```python
from agent_framework import SequentialBuilder

workflow = (
    SequentialBuilder()
    .participants([agent1, agent2, agent3])
    .build()
)

# Run
async for event in workflow.run_stream("your prompt"):
    if isinstance(event, WorkflowOutputEvent):
        print(event.data)
```

**Concurrent Workflow:**
```python
from agent_framework import ConcurrentBuilder

workflow = (
    ConcurrentBuilder()
    .participants([agent1, agent2, agent3])
    .build()
)

# Run and get outputs
result = await workflow.run("your prompt")
outputs = result.get_outputs()
```

**Pattern Selection:**
- **Sequential** → When order matters, agents build on each other
- **Concurrent** → When speed matters, independent tasks
- **Custom** → See Tutorial 07 for WorkflowBuilder
- **Magentic** → See Tutorial 07 for AI orchestration